<a href="https://colab.research.google.com/github/Nicolenki7/Customer-Sentiment-Analysis-Product-Improvement-Drivers/blob/main/Sentiment_Analysis_and_Satisfaction_Drivers_in_E_commerce_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import gc
import warnings

warnings.filterwarnings('ignore')

# --- 0. NLTK Configuration (Always necessary in Colab) ---
nltk.download('stopwords', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# --- 1. Define Parameters and File Path ---
FILE_PATH = 'Reviews.csv'
CHUNK_SIZE = 20000          # Reduced chunk size for safer processing
TARGET_SAMPLE_SIZE = 100000 # We aim for a final dataset of 100,000 rows
MAX_CHUNKS_TO_READ = TARGET_SAMPLE_SIZE // CHUNK_SIZE
VADER = SentimentIntensityAnalyzer()
STOPWORDS_EN = set(stopwords.words('english'))

# --- FIX for ParserError (Engine and quoting) ---
# Engine='python' handles complex encoding/quoting issues better than the default 'c' engine.
# quoting=3 (csv.QUOTE_NONE) helps prevent issues with quotes inside text.
CSV_READ_PARAMS = {
    'chunksize': CHUNK_SIZE,
    'usecols': ['Id', 'ProductId', 'Score', 'Text'],
    'engine': 'python',
    'encoding': 'utf-8',
    'on_bad_lines': 'skip'  # Skips malformed rows instead of crashing (Pandas 1.x+)
}

# --- 2. Text Cleaning and Scoring Function ---
def clean_and_score_chunk(df):
    """Applies cleaning and VADER scoring to a DataFrame chunk."""

    # Standardize column names
    df = df.rename(columns={'ProductId': 'product_id', 'Score': 'rating', 'Text': 'review_text'})
    df = df[['Id', 'product_id', 'rating', 'review_text']].copy()

    # Preprocessing
    df['clean_text'] = df['review_text'].astype(str).str.lower()
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))
    df['clean_text'] = df['clean_text'].apply(
        lambda x: ' '.join([word for word in x.split() if word not in STOPWORDS_EN])
    )

    # Apply VADER Sentiment Analysis
    df['compound_score'] = df['clean_text'].apply(lambda x: VADER.polarity_scores(x)['compound'])

    # Memory optimization
    df = df.drop(columns=['review_text', 'clean_text'])
    return df

# --- 3. Robust Chunking and Processing Loop ---
all_chunks = []
chunks_read = 0

try:
    print(f"Starting robust chunk processing. Target sample: {TARGET_SAMPLE_SIZE} reviews.")

    # Read CSV in chunks using the corrected parameters
    for chunk in pd.read_csv(FILE_PATH, **CSV_READ_PARAMS):

        if chunks_read >= MAX_CHUNKS_TO_READ:
            break

        print(f"Processing chunk {chunks_read + 1}/{MAX_CHUNKS_TO_READ}...")

        processed_chunk = clean_and_score_chunk(chunk)
        all_chunks.append(processed_chunk)

        del chunk, processed_chunk
        gc.collect()
        chunks_read += 1

    # Concatenate the final, large sample DataFrame
    df_reviews_robust = pd.concat(all_chunks, ignore_index=True)

except FileNotFoundError:
    print(f"ERROR: File not found at {FILE_PATH}. Please ensure Reviews.csv is uploaded.")
    df_reviews_robust = pd.DataFrame() # Create empty DF on failure

print("\n--- Robust Sample Processing Complete ---")
if not df_reviews_robust.empty:
    print(f"Final DataFrame size: {df_reviews_robust.shape[0]} rows.")
    print(df_reviews_robust.head())
else:
    print("DataFrame is empty due to previous error or file not found.")

Starting robust chunk processing. Target sample: 100000 reviews.
Processing chunk 1/5...
Processing chunk 2/5...
Processing chunk 3/5...
Processing chunk 4/5...
Processing chunk 5/5...

--- Robust Sample Processing Complete ---
Final DataFrame size: 100000 rows.
   Id  product_id  rating  compound_score
0   1  B001E4KFG0       5          0.9413
1   2  B00813GRG4       1         -0.1027
2   3  B000LQOCH0       4          0.8073
3   4  B000UA0QIQ       2          0.4404
4   5  B006K2ZZ7K       5          0.9468


In [3]:
import sqlite3
import pandas as pd
import gc

# Asumimos que df_reviews_robust (100k filas) ya está en memoria

# 1. Connection to SQLite Database (In-Memory for performance)
conn = sqlite3.connect(':memory:')

# 2. Dump the Robust DataFrame to an SQL Table
# Column names are now clean and standardized (product_id, rating, compound_score)
df_reviews_robust.to_sql(
    'reviews_sentiment',    # Table Name
    conn,
    if_exists='replace',
    index=False
)

# 3. Strategic SQL Query for Feature Engineering (IN ENGLISH)
sql_query_fe = """
SELECT
    Id,
    product_id,
    rating,
    compound_score,

    -- Feature 1: Sentiment Category (Positive/Negative/Neutral)
    CASE
        WHEN compound_score >= 0.05 THEN 'Positive'      -- Promoters
        WHEN compound_score <= -0.05 THEN 'Negative'     -- Detractors
        ELSE 'Neutral'                                   -- Passives
    END AS sentiment_category,

    -- Feature 2: High-Level Rating Category
    CASE
        WHEN rating IN (4, 5) THEN 'High_Rating'
        WHEN rating = 3 THEN 'Mid_Rating'
        ELSE 'Low_Rating'
    END AS rating_category

FROM
    reviews_sentiment;
"""

# Execute the query and load results into the final BI-ready DataFrame
df_final_bi_robust = pd.read_sql_query(sql_query_fe, conn)

# 4. Close the connection and clean memory
conn.close()
del df_reviews_robust
gc.collect()

print("--- SQL Feature Engineering on 100k Sample Complete ---")
print("New DataFrame (df_final_bi_robust) is ready for CSV export and BI.")
print(df_final_bi_robust.head())
print(f"\nDistribution of Sentiment (Should now show 'Neutral' values):")
print(df_final_bi_robust['sentiment_category'].value_counts())

--- SQL Feature Engineering on 100k Sample Complete ---
New DataFrame (df_final_bi_robust) is ready for CSV export and BI.
   Id  product_id  rating  compound_score sentiment_category rating_category
0   1  B001E4KFG0       5          0.9413           Positive     High_Rating
1   2  B00813GRG4       1         -0.1027           Negative      Low_Rating
2   3  B000LQOCH0       4          0.8073           Positive     High_Rating
3   4  B000UA0QIQ       2          0.4404           Positive      Low_Rating
4   5  B006K2ZZ7K       5          0.9468           Positive     High_Rating

Distribution of Sentiment (Should now show 'Neutral' values):
sentiment_category
Positive    89858
Negative     7987
Neutral      2155
Name: count, dtype: int64


In [4]:
# FINAL CSV EXPORT SCRIPT (ROBUST VERSION)

import os # For clean path printing

# 1. Define the Robust Output File Name
output_file_name = 'reviews_sentiment_TRANSFORMADO_ROBUSTO_100k.csv'

# 2. Export the final, BI-ready DataFrame
# df_final_bi_robust is the DataFrame from the previous step.
# index=False ensures a clean CSV file without unnecessary row numbers.
df_final_bi_robust.to_csv(output_file_name, index=False)

# 3. Confirmation
print(f"--- Final Export Successful ---")
print(f"The file '{output_file_name}' (100,000+ rows) is ready.")
print("Path for download/upload to BI tool (Looker/Tableau):")
print(f"Ruta completa: {os.getcwd()}/{output_file_name}")

--- Final Export Successful ---
The file 'reviews_sentiment_TRANSFORMADO_ROBUSTO_100k.csv' (100,000+ rows) is ready.
Path for download/upload to BI tool (Looker/Tableau):
Ruta completa: /content/reviews_sentiment_TRANSFORMADO_ROBUSTO_100k.csv
